Dependencies and pacakges
- python3
- pip pacakges - cirq, qsimcirq, cuquantum

In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

# Variational Quantum Eigensolver(VQE) using cuStateVec

Materials adopted from:
[https://quantumai.google/cirq/experiments/variational_algorithm](https://quantumai.google/cirq/experiments/variational_algorithm)

# Define the ansatz

Generate a random Ising Hamiltonian on a square grid

# $H=\sum_{\langle i,j \rangle} J_{i,j} Z_i Z_j + \sum_i h_iZ_i$

In [2]:
import cirq
import qsimcirq
import time
import copy
import random

In [3]:
import random
def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length - 1, length)
    # links within a column
    jc = rand2d(length, length - 1)
    return (h, jr, jc)

h, jr, jc = random_instance(3)

In [4]:
print(f'transverse fields: {h}')
print(f'row j fields: {jr}')
print(f'column j fields: {jc}')

transverse fields: [[1, -1, -1], [1, -1, 1], [-1, -1, 1]]
row j fields: [[-1, -1, -1], [-1, 1, -1]]
column j fields: [[-1, 1], [-1, 1], [-1, -1]]


### Prepare plus state

In [5]:
def prepare_plus_layer(length):
    for i in range(length):
        for j in range(length):
            yield cirq.H(cirq.GridQubit(i, j))

In [6]:
circuit = cirq.Circuit()
circuit.append(prepare_plus_layer(2))
print(circuit)

(0, 0): ───H───

(0, 1): ───H───

(1, 0): ───H───

(1, 1): ───H───


## X layer

In [7]:
def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""

    # Define the gate once and then re-use it for each Operation.
    rot = cirq.XPowGate(exponent=half_turns)

    # Create an X rotation Operation for each qubit in the grid.
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

In [8]:
# Create the circuit using the rot_x_layer generator
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


## Z layer

In [9]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

In [10]:
circuit = cirq.Circuit()
circuit.append(rot_z_layer(h, 0.1))
print(circuit)

(0, 0): ───Z^0.1───

(1, 0): ───Z^0.1───

(1, 2): ───Z^0.1───

(2, 2): ───Z^0.1───


## Entangling Layer

Apply a `cirq.CZPowGate` for the same parameter between all qubits where the coupling field term $J$ is $+1$. If the field is $-1$, apply `cirq.CZPowGate` conjugated by $X$ gates on all qubits.

In [11]:
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    cz_gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i + 1, j)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i, j + 1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

## Put it all together to generate the ansatz

5x5 grid - 25 qubits simulation

In [12]:
#define a 5x5 qubit ansatz

length=5
qubits=cirq.GridQubit.square(length)
h, jr, jc = random_instance(length)

circuit = cirq.Circuit()
circuit.append(prepare_plus_layer(length))
circuit.append(rot_z_layer(h,0.3))
circuit.append(rot_11_layer(jr,jc,0.3))
circuit.append(rot_x_layer(length, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))

In [13]:
print(circuit)

                       ┌───────────┐   ┌────────────────────┐   ┌──────────┐   ┌───────────────┐   ┌───────────────┐   ┌───────────────┐   ┌──────────┐           ┌──────┐
(0, 0): ───H───Z^0.3────@───────────────────────────────────────────────────────@───────────────────X^0.3────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M('x')───
                        │                                                       │                                                                                                                                                                                    │
(0, 1): ───H───X────────┼───────────────@────────────────────────X──────────────@^0.3───────────────@───────────────────X^0.3────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M────────
                        

# Run the simualation

In [15]:
# # Define the simulators
# # Default simulator
simulator = cirq.Simulator()

# cuQuantum enabled GPU simulator
ngpus = 1
qsim_options = qsimcirq.QSimOptions(
    max_fused_gate_size = 2,
    cpu_threads = 1,
    gpu_mode = ngpus,
    # use_sampler = True,
    disable_gpu = False
)

simulator_cuQ = qsimcirq.QSimSimulator(qsim_options)

TypeError: QSimOptions.__init__() got an unexpected keyword argument 'disable_gpu'

In [15]:
# Define the simulators
# Default simulator
simulator = cirq.Simulator()

# cuQuantum enabled GPU simulator
ngpus=1
qsim_options=qsimcirq.QSimOptions(
    max_fused_gate_size=2,
    cpu_threads=1,
    gpu_mode=ngpus
)
simulator_cuQ=qsimcirq.QSimSimulator(qsim_options)


In [16]:
# Default simulator
start=time.time()
results = simulator.run(circuit, repetitions=100)
print(results.histogram(key='x'))

Counter({29824339: 1, 19040736: 1, 16930850: 1, 18380753: 1, 22153384: 1, 16979652: 1, 4247448: 1, 1450257: 1, 1558912: 1, 954536: 1, 16961750: 1, 8880592: 1, 8570180: 1, 8888916: 1, 17248416: 1, 692809: 1, 10958865: 1, 11413633: 1, 2787025: 1, 19554481: 1, 21565528: 1, 9319016: 1, 985184: 1, 610545: 1, 17914135: 1, 4217360: 1, 18031732: 1, 14559553: 1, 18008216: 1, 19038656: 1, 19740228: 1, 17469777: 1, 18254130: 1, 17438020: 1, 19289424: 1, 17346800: 1, 9834791: 1, 2560436: 1, 3019856: 1, 30486704: 1, 2245764: 1, 2032884: 1, 6760352: 1, 19268817: 1, 1261968: 1, 21446800: 1, 479316: 1, 86768: 1, 594673: 1, 991400: 1, 9836373: 1, 17113776: 1, 4730012: 1, 5807060: 1, 5465280: 1, 18557106: 1, 17224020: 1, 328098: 1, 833745: 1, 8864713: 1, 4229824: 1, 19026352: 1, 17975972: 1, 3014996: 1, 19796548: 1, 18289737: 1, 10681408: 1, 18064597: 1, 2531464: 1, 21898312: 1, 721538: 1, 19800224: 1, 16943445: 1, 11037393: 1, 19858643: 1, 21146178: 1, 9836592: 1, 333504: 1, 11157904: 1, 1774940: 1, 30

In [17]:
print("CPU runtime: {:.2f}s".format(time.time() - start))

CPU runtime: 9.55s


In [18]:
# cuQuantum backend

start=time.time()
results = simulator_cuQ.run(circuit, repetitions=100)
print(results.histogram(key='x'))
print("GPU runtime: {:.2f}s".format(time.time() - start))

Counter({213024: 1, 595144: 1, 656497: 1, 665752: 1, 704680: 1, 725312: 1, 757936: 1, 886980: 1, 988234: 1, 1156736: 1, 1182856: 1, 1263670: 1, 1507792: 1, 1782402: 1, 2250800: 1, 2449119: 1, 2560928: 1, 2676912: 1, 2698377: 1, 2819275: 1, 2844380: 1, 3033760: 1, 3037584: 1, 4247744: 1, 4725005: 1, 4820226: 1, 4883649: 1, 4892112: 1, 5067120: 1, 5576021: 1, 5954567: 1, 6170804: 1, 7428673: 1, 8078544: 1, 9113715: 1, 9120945: 1, 9373856: 1, 10294368: 1, 10562208: 1, 10601369: 1, 10671440: 1, 10818256: 1, 11084992: 1, 11151941: 1, 11216022: 1, 14616457: 1, 14945232: 1, 15078050: 1, 15090080: 1, 15618198: 1, 16848565: 1, 16863530: 1, 16864329: 1, 16898057: 1, 16979153: 1, 16996189: 1, 17060016: 1, 17077056: 1, 17189267: 1, 17256512: 1, 17437024: 1, 17500832: 1, 17501332: 1, 17504470: 1, 17506544: 1, 17781973: 1, 17973680: 1, 18043073: 1, 18420673: 1, 18960817: 1, 18968721: 1, 19010880: 1, 19012192: 1, 19024001: 1, 19026241: 1, 19027152: 1, 19034432: 1, 19043403: 1, 19076276: 1, 19096777: 

# Calculate the energy expectation

In [19]:
import numpy as np

def energy_func(length, h, jr, jc):
    def energy(measurements):
        # Reshape measurement into array that matches grid shape.
        meas_list_of_lists = [measurements[i * length:(i + 1) * length]
                              for i in range(length)]
        # Convert true/false to +1/-1.
        pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

        tot_energy = np.sum(pm_meas * h)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
        return tot_energy
    return energy

def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(length, h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions

In [20]:
print(results.histogram(key='x', fold_func=energy_func(length, h, jr, jc)))
print(f'Value of the objective function {obj_func(results)}')

Counter({9: 18, 7: 16, 11: 10, 15: 7, 5: 7, -3: 6, -1: 6, 13: 6, 1: 5, 3: 4, 25: 3, 19: 2, 21: 2, 17: 2, -11: 2, -5: 1, -9: 1, -7: 1, 23: 1})
Value of the objective function 7.64


# Parametrize the ansatz to minimize the energy

In [21]:
import sympy

# Define the parameters symbolically
circuit = cirq.Circuit()
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')

# Define the circuit
circuit= cirq.Circuit()
circuit.append(prepare_plus_layer(length))
circuit.append(rot_z_layer(h,alpha))
circuit.append(rot_11_layer(jr, jc, beta))
circuit.append(rot_x_layer(length, gamma))
circuit.append(cirq.measure(*qubits, key='x'))

print(circuit)


               ┌───────────────────┐   ┌─────────────┐   ┌────────────────────────┐            ┌────────────────────────┐   ┌─────────────┐   ┌────────────────────┐   ┌─────────────┐             ┌────────┐
(0, 0): ───H───────────@────────────────X─────────────────────────────────────────────@─────────X────────────────────────────X^gamma───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M('x')───
                       │                                                              │                                                                                                                                                                                                            │
(0, 1): ───H────Z^alpha┼────────────────@─────────────────X───────────────────────────@^beta────X────────────────────────────@─────────────────X^gamma──────────────────────────────────────────────────

In [22]:
# Define the sweep size
sweep_size = 10
start=time.time()

sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=sweep_size)
         * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=sweep_size)
         * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=sweep_size))

# Execute the sweep using cuQuantum
results = simulator_cuQ.run_sweep(circuit, params=sweep, repetitions=100)
print("runtime: {:.2f}s".format(time.time() - start))

min = None
min_params = None
for result in results:
    value = obj_func(result)
    if min is None or value < min:
        min = value
        min_params = result.params
print(f'Minimum objective value is {min}.')


runtime: 1723.31s
Minimum objective value is -5.78.


In [23]:
# Execute the sweep using qsim
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
print("runtime: {:.2f}s".format(time.time() - start))

min = None
min_params = None
for result in results:
    value = obj_func(result)
    if min is None or value < min:
        min = value
        min_params = result.params
print(f'Minimum objective value is {min}.')


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/harryneo/anaconda3/envs/nvidia_quantum/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_49023/1522435971.py", line 2, in <module>
    results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/harryneo/anaconda3/envs/nvidia_quantum/lib/python3.11/site-packages/cirq/sim/simulator.py", line 72, in run_sweep
    return list(self.run_sweep_iter(program, params, repetitions))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/harryneo/anaconda3/envs/nvidia_quantum/lib/python3.11/site-packages/cirq/sim/simulator.py", line 103, in run_sweep_iter
    records = self._run(
              ^^^^^^^^^^
  File "/home/harryneo/anaconda3/envs/nvidia_quantum/lib/python3.11/site-packages/cirq/sim/simulator_base.py", l